# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


# Loading Index Files

In [5]:
df_eng_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_Mar262021\ESA_website_ENG_old_projects.csv", encoding='utf-8') # dtype fine
df_eng_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads\ESA_website_ENG_new_projects.csv", encoding='utf-8') # dtypes fine

df_fra_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_May062021_fra\ESA_website_FRA.csv", encoding='latin1') # dtype fine
df_fra_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\ESA_website_FRA_new_projects.csv", encoding='utf-8') # dtypes fine

df_eng_all_projects = pd.read_csv("G:\ESA_downloads\ESA_bundled_with_VCs\ESA_website_ENG.csv", encoding='utf-8')

C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df_eng_old_tables = df_eng_old_projects[df_eng_old_projects['Content Type'] == 'Table']

df_fra_old_tables = df_fra_old_projects[df_fra_old_projects['Type de contenu'] == 'Tableau']

In [12]:
print(len(df_eng_old_projects))
print(len(df_eng_new_projects))
print(len(df_fra_old_projects))
print(len(df_fra_new_projects))
print(len(df_eng_all_projects))
print(len(df_eng_old_projects) + len(df_eng_new_projects))
print(len(df_eng_old_tables))
print(len(df_fra_old_tables))

24285
644
24285
644
24929
24929
15151
15151


# Checking for duplicate rows

In [13]:
left_cols = ['Title', 'Document Number', 'PDF Page Number', 'Hearing order', 'File Name', 'Content Type']
len(df_eng_old_tables[left_cols].drop_duplicates(keep='first'))

15151

In [14]:
df_eng_old_tables[left_cols][df_eng_old_tables[left_cols].duplicated(keep='first')]

,Title,Document Number,PDF Page Number,Hearing order,File Name,Content Type


# Updating old and new index files with index file containing all projects and VCs

In [15]:
def merge_dfs_with_matching_specific_columns(left_df, right_df, left_on, right_on):
    """Merge two dataframes based on common columns and only keep rows where
    the values match.
    """
    merged_df = pd.merge(left_df, right_df, how='left', left_on=left_on, right_on=right_on, suffixes=('_x', ''))
    merged_df.drop(merged_df.filter(regex='_x$').columns, axis=1, inplace=True)
    return merged_df

left_cols = ['Title', 'Document Number', 'PDF Page Number', 'Hearing order', 'File Name', 'Content Type']
right_cols = ['Title', 'Document Number', 'PDF Page Number', 'Hearing order', 'File Name', 'Content Type']

df_eng_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_tables, df_eng_all_projects, left_on=left_cols, right_on=right_cols)
df_eng_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)

In [16]:
len(df_eng_old_merged.drop_duplicates(subset=['Title', 'Document Number', 'PDF Page Number'])) # Checking to make sure number of rows is the same as the original

15151

# Creating the Merged French Index

In [30]:
df_fra_old_projects.head()

,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,ID,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité
0,Figure 13.1-1 Pipeline EnCana Ekwan,Figure,Demande visant la construction et l'exploitati...,Ekwan,3/17/2003,EnCana Ekwan Pipeline Inc.,Gaz,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-1-2003,AXYS Environmental Consulting Ltd.,En exploitation,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,"Section 13.1 : Introduction, Section 13.2 : De...",1.0,"Terres, Air, Végétation",26,107.0,1.41,Oui,15151,NaN,NaN,NaN
1,Figure 13.3-1 Cadre de travail de l'ACEE,Figure,Demande visant la construction et l'exploitati...,Ekwan,3/17/2003,EnCana Ekwan Pipeline Inc.,Gaz,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-1-2003,AXYS Environmental Consulting Ltd.,En exploitation,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,"Section 13.1 : Introduction, Section 13.2 : De...",1.0,"Terres, Air, Végétation",41,107.0,1.41,Oui,15152,NaN,NaN,NaN
2,Figure 13.4-1 Normales de température mesurées...,Figure,Demande visant la construction et l'exploitati...,Ekwan,3/17/2003,EnCana Ekwan Pipeline Inc.,Gaz,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-1-2003,AXYS Environmental Consulting Ltd.,En exploitation,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,"Section 13.1 : Introduction, Section 13.2 : De...",1.0,"Terres, Air, Végétation",44,107.0,1.41,Oui,15153,NaN,NaN,NaN
3,Figure 13.4-2 Pluie mensuelle moyenne et nombr...,Figure,Demande visant la construction et l'exploitati...,Ekwan,3/17/2003,EnCana Ekwan Pipeline Inc.,Gaz,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-1-2003,AXYS Environmental Consulting Ltd.,En exploitation,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,"Section 13.1 : Introduction, Section 13.2 : De...",1.0,"Terres, Air, Végétation",45,107.0,1.41,Oui,15154,NaN,NaN,NaN
4,Figure 13.4-3 Chute de neige mensuelle moyenne...,Figure,Demande visant la construction et l'exploitati...,Ekwan,3/17/2003,EnCana Ekwan Pipeline Inc.,Gaz,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Colombie-Britannique",GH-1-2003,AXYS Environmental Consulting Ltd.,En exploitation,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,"Section 13.1 : Introduction, Section 13.2 : De...",1.0,"Terres, Air, Végétation",46,107.0,1.41,Oui,15155,NaN,NaN,NaN


In [32]:
labels_list = ['Landscape, terrain, and weather', 
                'Soil',
                'Plants',
                'Water',
                'Fish',
                'Wetlands',
                'Wildlife',
                'Species at Risk',
                'Greenhouse gas emissions',
                'Air emissions',
                'Noise',
                'Electricity and electromagnetism',
                'Proximity to people',
                'Archaeological, paleontological, historical, and culturally significant sites and resources',
                'Human access to boats and waterways',
                'Indigenous land, water, and air use',
                'Impact to social and cultural well-being',
                'Impact to human health and viewscapes',
                'Social, cultural, economic infrastructure and services',
                'Economic Offsets and Impact',
                'Environmental Obligations',
                'Treaty and Indigenous Rights']

labels_list_fr = ['Paysage, terrain et conditions météorologiques',
                    'Sol',
                    'Plantes',
                    'Eau',
                    'Poissons',
                    'Milieux humides',
                    'Faune',
                    'Espèces en péril',
                    'Émissions de gaz à effet de serre',
                    'Émissions atmosphériques',
                    'Bruit',
                    'Électricité et électromagnétisme',
                    'Lieux habités à proximité',
                    'Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel',
                    'Activités à bord d’embarcation ou sur des voies navigables',
                    'Utilisation des terres, de l’eau et du plein air par les Autochtones',
                    'Incidences sur le bien-être social et culturel',
                    'Incidence sur la santé humaine et le panorama',
                    'Infrastructure et services sociaux, culturels et économiques',
                    'Compensations et répercussions économiques',
                    'Obligations en matière d’environnement',
                    'Droits ancestraux et issus de traités']

In [52]:
def add_empty_list_of_columns_to_df(df, list_of_columns):
    for column in list_of_columns:
        df[column] = float(0)
    return df

df_fra_old_projects = add_empty_list_of_columns_to_df(df_fra_old_projects, labels_list_fr)
df_fra_new_projects = add_empty_list_of_columns_to_df(df_fra_new_projects, labels_list_fr)

In [53]:
df_fra_new_projects.head()

,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,ID,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité,Data ID,"Paysage, terrain et conditions météorologiques",Sol,Plantes,Eau,Poissons,Milieux humides,Faune,Espèces en péril,Émissions de gaz à effet de serre,Émissions atmosphériques,Bruit,Électricité et électromagnétisme,Lieux habités à proximité,"Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel",Activités à bord d’embarcation ou sur des voies navigables,"Utilisation des terres, de l’eau et du plein air par les Autochtones",Incidences sur le bien-être social et culturel,Incidence sur la santé humaine et le panorama,"Infrastructure et services sociaux, culturels et économiques",Compensations et répercussions économiques,Obligations en matière d’environnement,Droits ancestraux et issus de traités
0,Les tableaux 10.14 et 10.16 présentent des lis...,Tableau,Demande visant le projet de livraison parcours...,Livraison parcours ouest 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gaz,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Demande présentée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,NaN,7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HA...,NaN,NaN,309,612,NaN,NaN,505,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_les-tableaux-10-14-et-1...,True,4003703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Les tableaux 13.28, 13.29 et 13.30 fournissent...",Tableau,Demande visant le projet de livraison parcours...,Livraison parcours ouest 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gaz,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Demande présentée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,NaN,7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HA...,NaN,NaN,476,612,NaN,NaN,118,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_les-tableaux-13-28-13-2...,True,4003703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Les tableaux 7.21, 7.22 et 7.23 présentent les...",Tableau,Demande visant le projet de livraison parcours...,Livraison parcours ouest 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gaz,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Demande présentée,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,NaN,7.0 ÉVALUATION DES EFFETS SUR LA FAUNE ET L'HA...,NaN,NaN,115,612,NaN,NaN,506,/projects/wstpthdlvr2023.zip,/tables/wstpthdlvr2023_les-tableaux-7-21-7-22-...,True,4003703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TABLEAU 1B MESURES D'ATTÉNUATION PROPRES AU PR...,Tableau,Demande visant le projet de livraison parcours...,Livraison parcours ouest 2022,6/1/2020,NOVA Gas Transmission Ltd.,Gaz,C08618-3 Attachment CER 1.4-1_Part 1 of 6_ESA_...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A7I8X1,https://apps.cer-rec.gc.ca/REG

In [61]:
left_cols = ['Document Number', 'Hearing order', 'Consultant Name', 'PDF Page Number', 'File Name', 'Content Type']
left_cols.extend(labels_list)
right_cols = ['Numéro de document', 'Ordonnance d\'audience', 'Nom du consultant', 'Numéro de page PDF', 'Nom de fichier', 'Type de contenu']
right_cols.extend(labels_list_fr)

In [62]:
print(left_cols)

['Document Number', 'Hearing order', 'Consultant Name', 'PDF Page Number', 'File Name', 'Content Type', 'Landscape, terrain, and weather', 'Soil', 'Plants', 'Water', 'Fish', 'Wetlands', 'Wildlife', 'Species at Risk', 'Greenhouse gas emissions', 'Air emissions', 'Noise', 'Electricity and electromagnetism', 'Proximity to people', 'Archaeological, paleontological, historical, and culturally significant sites and resources', 'Human access to boats and waterways', 'Indigenous land, water, and air use', 'Impact to social and cultural well-being', 'Impact to human health and viewscapes', 'Social, cultural, economic infrastructure and services', 'Economic Offsets and Impact', 'Environmental Obligations', 'Treaty and Indigenous Rights']


In [66]:
right_cols = ['Document Number', 'Hearing order', 'Consultant Name', 'PDF Page Number', 'File Name', 'Content Type']
right_cols.extend(labels_list)
left_cols = ['Numéro de document', 'Ordonnance d\'audience', 'Nom du consultant', 'Numéro de page PDF', 'Nom de fichier', 'Type de contenu']
left_cols.extend(labels_list_fr)

In [68]:
df_fra_old_merged = merge_dfs_with_matching_specific_columns(df_fra_old_projects, df_eng_old_merged, left_on=left_cols, right_on=right_cols)
df_fra_new_merged = merge_dfs_with_matching_specific_columns(df_fra_new_projects, df_eng_new_merged, left_on=left_cols, right_on=right_cols)

In [63]:
df_fra_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_merged, df_fra_old_projects, left_on=left_cols, right_on=right_cols)
df_fra_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_merged, df_fra_new_projects, left_on=left_cols, right_on=right_cols)

In [70]:
df_fra_old_merged.tail()

,Titre,Type de contenu,Nom de la demande,Nom abrégé de la demande,Dépôt de la demande,Nom de la société,Produit de base,Nom de fichier,URL du dossier de l'ÉES,Numéro de document,URL de téléchargement PDF,Type de demande (Loi sur l'Office national de l'énergie),Emplacement du pipeline,Ordonnance d'audience,Nom du consultant,État d'avancement,Instruments réglementaires,URL de la demande,URL de la décision,Sections de l'EES,Index des sections de l'ÉES,Sujets des sections de l'ÉES,Numéro de page PDF,Nombre de pages PDF,Taille PDF,Aperçu PDF,Chemin d'accès pour télécharger le projet,Chemin d'accès pour télécharger le tableau,Bonne qualité,"Paysage, terrain et conditions météorologiques",Sol,Plantes,Eau,Poissons,Milieux humides,Faune,Espèces en péril,Émissions de gaz à effet de serre,Émissions atmosphériques,Bruit,Électricité et électromagnétisme,Lieux habités à proximité,"Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel",Activités à bord d’embarcation ou sur des voies navigables,"Utilisation des terres, de l’eau et du plein air par les Autochtones",Incidences sur le bien-être social et culturel,Incidence sur la santé humaine et le panorama,"Infrastructure et services sociaux, culturels et économiques",Compensations et répercussions économiques,Obligations en matière d’environnement,Droits ancestraux et issus de traités,Title,Content Type,File Name,Document Number,Hearing order,PDF Page Number,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,ESA Folder URL,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality
24280,Tableau A8-1 Autorités de signalement des reje...,Tableau,Demande relative au pipeline Keystone XL,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Pétrole,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Demande approuvée,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,« Annexe A : Plan de protection environnemental,33.0,Plan de protection de l'environnement,103,118.0,3.59,Non,NaN,NaN,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24281,Tableau A8-2 Critères de signalement des rejets,Tableau,Demande relative au pipeline Keystone XL,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Pétrole,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1...,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Projets de grande envergure (plus de 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Demande approuvée,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Élément/Aff...,https://apps.cer-rec.gc.ca/REGDOCS/%C3%89l%C3%...,Annexe A : 